Make sure you update the **STUDENT_TOKEN**, and then run this block. You should use FIRSTNAME-SURNAME(S) as the TOKEN.

The code in this block can be safely ignored.

In [ ]:

import requests
import pprint
import json
import time
from abc import abstractmethod
from collections import deque
import copy
import typing
import datetime
from queue import PriorityQueue
import math


def get_problem(problem_id):
  """Get a problem from the server."""

  r = requests.get('https://emarchiori.eu.pythonanywhere.com/get-problem?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id))
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  return r.json()


def submit_answer(problem_id, answer):
  """Submit an answer to the server, for validation.

  Note: there is no need to use this for this assignment.
  """

  r = requests.get('https://emarchiori.eu.pythonanywhere.com/submit-answer?TOKEN=%s&problem=%s' % (STUDENT_TOKEN, problem_id), json = answer)
  if not r.status_code == 200:
    print('\033[91m' + str(r.status_code))
  result = r.json()['result']
  if result == 'PASSED':
    print('\033[92m' + result)
  else:
    print('\033[91m' + result)


class State:
  """The abstract state for a problem.

  Remember to implement __str__, __eq__ and __hash__.
  """


class Action:
  """An action in a search problem."""

  def __init__(self, name: str, end: State, cost: int):
    self.name = name
    self.end = end
    self.cost = cost


class Node:
  """A node in a search tree."""

  def __init__(self, state: State, parent: "Node", action: Action, path_cost: int) -> None:
    self.state = state
    self.parent = parent
    self.action = action
    self.path_cost = path_cost


class Problem:
  """The abstract class for a problem.

  You should subclass this and implement the methods."""

  @abstractmethod
  def get_initial_state(self) -> State:
    pass

  @abstractmethod
  def is_goal(self, state: State) -> bool:
    pass

  @abstractmethod
  def actions(self, parent: State) -> list[Action]:
    pass

  @abstractmethod
  def heuristic(self, state) -> int:
    pass


class Solution:
  """The abstract class for a solution."""

  def __init__(self, reached: int, branching: float, time: float, node: Node == None):
    self.time = time
    self.reached = reached
    self.branching = branching
    self.node = node


  def to_string(self, show_state: bool = False) -> str:
    if not self.node:
      time.time
      return f"Reached: {self.reached}\nBranching: {self.branching}\nTime: {datetime.timedelta(seconds=self.time)}\nFAILED"
    if not show_state:
      path = []
      node = self.node
      while node.parent:
        path.insert(0, node.action.name)
        node = node.parent
      path_str = " -> ".join(path)
    else:
      state_path = []
      node = self.node
      while node.parent:
        state_path.insert(0, node.action.name + " > \n" + str(node.state))
        node = node.parent
      state_path.insert(0, str(node.state))
      path_str = " -> \n".join(state_path)
    return f"Cost: {self.node.path_cost}\nReached: {self.reached}\nBranching: {self.branching}\nTime: {datetime.timedelta(seconds=self.time)}\nPath:\n{path_str}"


def expand(problem: Problem, parent: Node):
  """Expand a node based on all possible actions."""

  start_state = parent.state
  for action in problem.actions(start_state):
      end_state = action.end
      cost = parent.path_cost + action.cost
      yield Node(end_state, parent, action, cost)


In [ ]:
# Breath-First Search

def breath_first_search(problem):
  """Breath-First Search implementation"""

  # Initial node is created from the initial state of the problem.
  initial_state = problem.get_initial_state()
  initial_node = Node(initial_state, None, None, 0)

  # We return immediately if the inital state is an end state
  # This is especially necessary given that we will then do an early goal test
  if problem.is_goal(initial_state):
    return Solution(0, 0, 0, initial_node)

  # Initialize all data structures.
  frontier = deque([initial_node])
  reached = set()
  branching = []

  start_time = time.perf_counter()

  while frontier:
    # Turning "popleft" into "pop" makes this algo DFS. However, you probably
    # also want to remove "reached" to reduce memory (otherwise you still keep)
    # a copy of each state.
    parent = frontier.popleft()

    b = 0 # Used to track the branching for each node
    for child in expand(problem, parent):
      s = child.state
      if s in reached:
        continue
      reached.add(s)
      frontier.append(child)
      b += 1

      if problem.is_goal(s): # Early goal test
        branching.append(b)
        return Solution(len(reached), sum(branching) / len(branching), time.perf_counter() - start_time, child)
    branching.append(b)

  return Solution(len(reached), sum(branching) / len(branching), time.perf_counter() - start_time, None)

In [ ]:
class VacuumState(State):
    def __init__(self, dirt:list[bool], pos:int):
        self.dirt = dirt
        self.pos = pos

    def __str__(self):
        return f"dirt: {self.dirt}, pos: {self.pos}"

    def __eq__(self, other):
        return self.dirt == other.dirt and self.pos == other.pos

    def __hash__(self):
        return hash(str(self.dirt) + str(self.pos))


class Vacuum(Problem):
  def get_initial_state(self) -> State:
    return VacuumState([True, True], 0)

  def actions(self, state) -> Action:
    actions = []
    actions.append(Action("left", VacuumState(state.dirt.copy(), max(state.pos - 1, 0)), 1))
    actions.append(Action("right", VacuumState(state.dirt.copy(), min(state.pos + 1, 1)), 1))
    dirt = state.dirt.copy()
    dirt[state.pos] = False
    actions.append(Action("suck", VacuumState(dirt, state.pos), 1))
    return actions

  def is_goal(self, state) -> bool:
    return not any(state.dirt)

  def heuristic(self, state) -> int:
    return 0

print(breath_first_search(Vacuum()).to_string(True))

Cost: 3
Reached: 7
Branching: 1.1666666666666667
Time: 0:00:00.000142
Path:
dirt: [True, True], pos: 0 -> 
suck > 
dirt: [False, True], pos: 0 -> 
right > 
dirt: [False, True], pos: 1 -> 
suck > 
dirt: [False, False], pos: 1


In [ ]:
class HanoiState(State):
  def __init__(self, towers):
    self.towers = towers

  def __str__(self):
    return ", ".join([str(t) for t in self.towers])

  def __hash__(self):
    return hash(tuple([tuple(t) for t in self.towers]))

  def __eq__(self, other):
    return self.towers == other.towers

  def move(self, from_tower, to_tower) -> "HanoiState":
    towers = copy.deepcopy(self.towers)
    towers[to_tower].append(towers[from_tower].pop())
    return HanoiState(towers)

class Hanoi(Problem):
  def get_initial_state(self) -> State:
    return HanoiState([[9, 8, 7, 6, 5, 4, 3, 2, 1], [], []])

  def actions(self, state) -> Action:
    actions = []

    for i in range(3):
      for j in range(3):
        if i != j and len(state.towers[i]) > 0 and (len(state.towers[j]) == 0 or state.towers[i][-1] < state.towers[j][-1]):
          actions.append(Action(f"move_{i}_{j}", state.move(i, j), 1))

    return actions

  def is_goal(self, state) -> bool:
    return len(state.towers[0]) == 0 and len(state.towers[1]) == 0

  def heuristic(self, state) -> int:
    return 0


print(breath_first_search(Hanoi()).to_string(False))

Cost: 511
Reached: 19513
Branching: 1.0223188557657044
Time: 0:00:01.980843
Path:
move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1

In [ ]:
class SlidingState(State):
  def __init__(self, board):
    self.board = board

  def __str__(self):
    return "--------\n" + "\n".join([str(l) for l in self.board]) + "\n--------"

  def __hash__(self):
    return hash(tuple([tuple(l) for l in self.board]))

  def __eq__(self, other):
    return self.board == other.board

  def swap(self, from_x, from_y, to_x, to_y):
    board = copy.deepcopy(self.board)
    board[from_x][from_y], board[to_x][to_y] = board[to_x][to_y], board[from_x][from_y]
    return SlidingState(board)


class Sliding(Problem):
  def __init__(self,
               initial_state: SlidingState = SlidingState([[0, 2, 3], [4, 1, 6], [7, 5, 8]]),
               final_state: SlidingState = SlidingState([[1, 2, 3], [4, 5, 6], [7, 8, 0]])) -> None:
    self.initial_state = initial_state
    self.final_state = final_state

  def get_initial_state(self):
    return self.initial_state

  def actions(self, state):
    actions = []

    for i in range(3):
      for j in range(3):
        if state.board[i][j] == 0:
          if i > 0:
            actions.append(Action("up", state.swap(i, j, i - 1, j), 1))
          if i < len(state.board) - 1:
            actions.append(Action("down", state.swap(i, j, i + 1, j), 1))
          if j > 0:
            actions.append(Action("left", state.swap(i, j - 1, i, j), 1))
          if j < len(state.board) - 1:
            actions.append(Action("right", state.swap(i, j + 1, i, j), 1))
    return actions

  def is_goal(self, state):
    return state.board == self.final_state.board

  def heuristic(self, state) -> int:
    return 0

problem = get_problem('sliding_tiles')
pprint.pprint(problem)

def json_to_board(state):
  flat = list(map(lambda n: int(n), state.split(',')))
  return [flat[i:i+3] for i in range(0,len(flat), 3)]

initial = SlidingState(json_to_board(problem['initial-state']))
final = SlidingState(json_to_board(problem['final-state']))

print(breath_first_search(Sliding(initial, final)).to_string(False))


{'explanation': '3x3 grid of tiles, with one missing. Tiles can only move from '
                'an adjacent place to the empty one (0).',
 'final-state': '1,2,3,4,5,6,7,8,0',
 'initial-state': '1,6,0,4,3,2,7,5,8',
 'path-format': '([UDLR](-[UDLR])*)?'}
Cost: 8
Reached: 183
Branching: 1.7102803738317758
Time: 0:00:00.009802
Path:
down -> left -> up -> right -> down -> left -> down -> right


In [ ]:
problem = get_problem('labyrinth-small') ## You can also try labyrinth-big or labyrinth-massive (but the last one is very big and slow)
pprint.pprint(problem)

print([list(s) for s in problem['map'].split('\n')])

{'explanation': 'Natigate the mace, from S to E. # represents walls, . empty '
                'spaces. Allowed moves: U, D, L, R',
 'final-pos': 'E',
 'initial-pos': 'S',
 'key-pos': 'K',
 'map': '#####################\n'
        '#S#...#.#.#.........#\n'
        '#.#.###.#.#.#.#.#####\n'
        '#.......#.#.#.#.....#\n'
        '#.#.#####.#.#####.###\n'
        '#.#.........#...#.#.#\n'
        '#.#####.#.###.#####.#\n'
        '#.....#.#.......#.#.#\n'
        '#.###.#.#####.###.#.#\n'
        '#...#.#.....#...#...#\n'
        '#.#.###.#.#########.#\n'
        '#.#...#.#.#...#.#...#\n'
        '###.#####.###.#.###.#\n'
        '#...#...............#\n'
        '#.#.#.#.#.#####.#.###\n'
        '#.#.#.#.#.#.....#...#\n'
        '#.#####.#######.#.#.#\n'
        '#...#.#.....#...#.#.#\n'
        '#.###.#.#####.###.#.#\n'
        '#.....#...#.....#.#E#\n'
        '#####################',
 'path-format': '[UDLR](-[UDLR])*'}
[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', 

In [ ]:
class HanoiState(State):
  def __init__(self, towers):
    self.towers = towers

  def __str__(self):
    return ", ".join([str(t) for t in self.towers])

  def __hash__(self):
    return hash(tuple([tuple(t) for t in self.towers]))

  def __eq__(self, other):
    return self.towers == other.towers

  def move(self, from_tower, to_tower) -> "HanoiState":
    towers = copy.deepcopy(self.towers)
    towers[to_tower].append(towers[from_tower].pop())
    return HanoiState(towers)

class Hanoi(Problem):
  def get_initial_state(self) -> State:
    return HanoiState([[9, 8, 7, 6, 5, 4, 3, 2, 1], [], []])

  def actions(self, state) -> Action:
    actions = []

    for i in range(3):
      for j in range(3):
        if i != j and len(state.towers[i]) > 0 and (len(state.towers[j]) == 0 or state.towers[i][-1] < state.towers[j][-1]):
          actions.append(Action(f"move_{i}_{j}", state.move(i, j), 1))

    return actions

  def is_goal(self, state) -> bool:
    return len(state.towers[0]) == 0 and len(state.towers[1]) == 0

  def heuristic(self, state) -> int:
    return 0


print(breath_first_search(Hanoi()).to_string(False))

Cost: 511
Reached: 19513
Branching: 1.0223188557657044
Time: 0:00:01.616507
Path:
move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_2_0 -> move_1_0 -> move_2_1 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1_0 -> move_1_2 -> move_0_2 -> move_1_0 -> move_2_1 -> move_2_0 -> move_1_0 -> move_1_2 -> move_0_2 -> move_0_1 -> move_2_1 -> move_0_2 -> move_1

In [ ]:
#JUGS

class JugsState(State):
    def __init__(self, jug_a, jug_b, jug_c):
        self.jug_a = jug_a
        self.jug_b = jug_b
        self.jug_c = jug_c

    def __str__(self):
        return f"Jug A: {self.jug_a}, Jug B: {self.jug_b}, Jug C: {self.jug_c}"

    def __eq__(self, other):
        return (self.jug_a == other.jug_a and self.jug_b == other.jug_b and
                self.jug_c == other.jug_c)

    def __hash__(self):
        return hash((self.jug_a, self.jug_b, self.jug_c))

class JugsProblem(Problem):
    def __init__(self):
        self.capacity_a = 8
        self.capacity_b = 5
        self.capacity_c = 3

    def get_initial_state(self) -> State:
        return JugsState(8, 0, 0)

    def actions(self, state) -> Action:
      actions = []

      jugs = [state.jug_a, state.jug_b, state.jug_c]
      capacities = [self.capacity_a, self.capacity_b, self.capacity_c]

      for i in range(3):  # source
          for j in range(3):  # desttination
              if i != j and jugs[i] > 0: # if not the same
                  pour_amount = min(jugs[i], capacities[j] - jugs[j])
                  if pour_amount > 0:
                      new_jugs = jugs.copy()
                      new_jugs[i] -= pour_amount
                      new_jugs[j] += pour_amount
                      new_state = JugsState(new_jugs[0], new_jugs[1], new_jugs[2])
                      actions.append(Action(f"From {i} to {j}", new_state, 1))

      return actions

    def heuristic(self, state) -> int: # it will be usefull with A*
        return 0

    def is_goal(self, state) -> bool:
        return state.jug_a == 4 and state.jug_b == 4 and state.jug_c == 0

print(breath_first_search(JugsProblem()).to_string(True))

Cost: 7
Reached: 15
Branching: 1.0714285714285714
Time: 0:00:00.000368
Path:
Jug A: 8, Jug B: 0, Jug C: 0 -> 
From 0 to 1 > 
Jug A: 3, Jug B: 5, Jug C: 0 -> 
From 1 to 2 > 
Jug A: 3, Jug B: 2, Jug C: 3 -> 
From 2 to 0 > 
Jug A: 6, Jug B: 2, Jug C: 0 -> 
From 1 to 2 > 
Jug A: 6, Jug B: 0, Jug C: 2 -> 
From 0 to 1 > 
Jug A: 1, Jug B: 5, Jug C: 2 -> 
From 1 to 2 > 
Jug A: 1, Jug B: 4, Jug C: 3 -> 
From 2 to 0 > 
Jug A: 4, Jug B: 4, Jug C: 0


In [ ]:
#basic labyrinth without heuristic
class LabyrinthState(State):
    def __init__(self, position):
        self.position = position

    def __str__(self):
        return f"Position: {self.position}"

    def __eq__(self, other):
        return self.position == other.position

    def __hash__(self):
        return hash(self.position)

class Labyrinth(Problem):
    def __init__(self, labyrinth_map: str, start: tuple[int, int], end: tuple[int, int]):
        self.labyrinth_map = [list(s) for s in labyrinth_map.split('\n')]
        self.start = start
        self.end = end
        self.map_height = len(self.labyrinth_map)
        self.map_width = len(self.labyrinth_map[0])

    def get_initial_state(self) -> State:
        return LabyrinthState(self.start)

    def actions(self, state) -> list[Action]:
        actions = []
        row, col = state.position

        # Up, Down, Left, Right
        moves = {
            "U": (row - 1, col),
            "D": (row + 1, col),
            "L": (row, col - 1),
            "R": (row, col + 1)
        }

        for move_name, (new_row, new_col) in moves.items():
            if self.is_valid_position(new_row, new_col):
                new_state = LabyrinthState((new_row, new_col))
                actions.append(Action(move_name, new_state, 1))

        return actions

    def is_valid_position(self, row, col) -> bool:
        if 0 <= row < self.map_height and 0 <= col < self.map_width:
            return self.labyrinth_map[row][col] != '#'
        return False

    def heuristic(self, state) -> int:
        return 0

    def is_goal(self, state) -> bool:
      return state.position == self.end


def find_positions(labyrinth_map) -> tuple[tuple[int, int], tuple[int, int]]:
  """Finding the positions of start 'S' and end 'E' from the map."""
  start_pos = end_pos = None
  for row_index, row in enumerate(labyrinth_map):
      for col_index, char in enumerate(row):
          if char == 'S':
            start_pos = (row_index, col_index)
          elif char == 'E':
            end_pos = (row_index, col_index)
  return start_pos, end_pos


problem = get_problem('labyrinth-big') # here we can change the size of the labyritnh
start_pos, end_pos = find_positions(problem['map'].split('\n'))

print(breath_first_search(Labyrinth(problem['map'], start_pos, end_pos)).to_string(False))
pprint.pprint(problem)

#print(A_Star_search(Labyrinth(problem['map'], start_pos, end_pos)).to_string(False))
#pprint.pprint(problem)


Cost: 204
Reached: 4997
Branching: 1.0
Time: 0:00:00.035952
Path:
D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> R -> R -> U -> U -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> R -> R -> U -> U -> R -> R -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> D -> D -> R -> R -> D -> D -> R -> R -> R -> R -> R ->

In [ ]:
from queue import PriorityQueue

def A_Star_search(problem):
    """A* Search Algorythm"""

    # Initial node is created from the initial state of the problem.
    initial_state = problem.get_initial_state()
    initial_node = Node(initial_state, None, None, 0)

    # Initialize frontier as a priority queue containing the initial state with priority 0
    frontier = PriorityQueue()
    count = 0  # Counter to break ties in priority queue
    frontier.put((0, count, initial_node))

    # Table to store reached states and their corresponding node information
    reached = {initial_state: initial_node}

    # Initialize solution as failure
    solution = None

    # Branching data
    total_expansions = 0
    total_children = 0

    start_time = time.perf_counter()

    while not frontier.empty():
        # Get the node with the lowest priority (f(n) = g(n) + h(n))
        _, _, parent = frontier.get()

        # If solution already found and the current node is the same as the solution, terminate
        if solution and parent.state == solution.state:
            break

        # Expand the current node
        children_count = 0
        for child in expand(problem, parent):
            s = child.state
            # If the state is not in reached or child path cost is cheaper, update the reached table
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child

                # Only add to the frontier if it's a new state or a better path to an existing state
                f_value = child.path_cost + problem.heuristic(child.state)
                count += 1
                frontier.put((f_value, count, child))
                children_count += 1

                # If the child state is a goal and is cheaper than the current solution, update solution
                if problem.is_goal(s) and (not solution or child.path_cost < solution.path_cost):
                    solution = child

        # Update branching statistics only when expanding a node
        total_expansions += 1
        total_children += children_count

    # Calculate average branching factor
    branching = total_children / total_expansions if total_expansions > 0 else 0

    # Return the solution (it can be a failure if no solution is found)
    return Solution(len(reached), branching, time.perf_counter() - start_time, solution)


In [ ]:
# EX 2 Sliding-tile with Mannhatan heuristic


class SlidingState(State):
  def __init__(self, board):
    self.board = board
  def __str__(self):
    return "--------\n" + "\n".join([str(l) for l in self.board]) + "\n--------"

  def __hash__(self):
    return hash(tuple([tuple(l) for l in self.board]))

  def __eq__(self, other):
    return self.board == other.board

  def swap(self, from_x, from_y, to_x, to_y):
    board = copy.deepcopy(self.board)
    board[from_x][from_y], board[to_x][to_y] = board[to_x][to_y], board[from_x][from_y]
    return SlidingState(board)


class Sliding(Problem):
  def __init__(self,
               initial_state: SlidingState = SlidingState([[1, 2, 0], [3, 4, 6], [7, 5, 8]]),
               final_state: SlidingState = SlidingState([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
               ) -> None:
    self.initial_state = initial_state
    self.final_state = final_state

  def get_initial_state(self):
    return self.initial_state

  def actions(self, state):
    actions = []

    for i in range(3):
      for j in range(3):
        if state.board[i][j] == 0:
          if i > 0:
            actions.append(Action("up", state.swap(i, j, i - 1, j), 1))
          if i < len(state.board) - 1:
            actions.append(Action("down", state.swap(i, j, i + 1, j), 1))
          if j > 0:
            actions.append(Action("left", state.swap(i, j - 1, i, j), 1))
          if j < len(state.board) - 1:
            actions.append(Action("right", state.swap(i, j + 1, i, j), 1))
    return actions

  def is_goal(self, state):
    return state.board == self.final_state.board

  def heuristic(self, state):
    """Manhattan distance heuristic, how far is a value to its goal place"""
    distance = 0
    for i in range(3):
      for j in range(3):
        value = state.board[i][j]
        if value != 0:
          # coordinates of the current value
          target_x = (value - 1) // 3
          target_y = (value - 1) % 3
          # Manhattan distance
          distance += abs(i - target_x) + abs(j - target_y)
    return distance


problem = get_problem('sliding_tiles')
pprint.pprint(problem)

def json_to_board(state):
  flat = list(map(lambda n: int(n), state.split(',')))
  return [flat[i:i+3] for i in range(0,len(flat), 3)]

initial = SlidingState(json_to_board(problem['initial-state']))
final = SlidingState(json_to_board(problem['final-state']))


print(breath_first_search(Sliding(initial, final)).to_string(False))
print(A_Star_search(Sliding(initial, final)).to_string(False))



{'explanation': '3x3 grid of tiles, with one missing. Tiles can only move from '
                'an adjacent place to the empty one (0).',
 'final-state': '1,2,3,4,5,6,7,8,0',
 'initial-state': '1,2,3,7,4,5,0,8,6',
 'path-format': '([UDLR](-[UDLR])*)?'}
Cost: 4
Reached: 23
Branching: 1.9166666666666667
Time: 0:00:00.000542
Path:
up -> right -> right -> down
Cost: 4
Reached: 10
Branching: 2.25
Time: 0:00:00.000301
Path:
up -> right -> right -> down


In [ ]:
class SlidingState:
    def __init__(self, board):
        self.board = board

    def __str__(self):
        return "--------\n" + "\n".join([str(l) for l in self.board]) + "\n--------"

    def __hash__(self):
        return hash(tuple([tuple(l) for l in self.board]))

    def __eq__(self, other):
        return self.board == other.board


    def swap(self, from_x, from_y, to_x, to_y):
        board = copy.deepcopy(self.board)
        board[from_x][from_y], board[to_x][to_y] = board[to_x][to_y], board[from_x][from_y]
        return SlidingState(board)


class Sliding:
    def __init__(self,
                 initial_state: SlidingState = SlidingState([[1, 2, 0], [3, 4, 6], [7, 5, 8]]),
                 final_state: SlidingState = SlidingState([[1, 2, 3], [4, 5, 6], [7, 8, 0]]),
                 heuristic_type: str = 'manhattan') -> None:
        self.initial_state = initial_state
        self.final_state = final_state
        self.heuristic_type = heuristic_type

    def get_initial_state(self):
        return self.initial_state

    def actions(self, state):
        actions = []

        for i in range(3):
            for j in range(3):
                if state.board[i][j] == 0:
                    if i > 0:
                        actions.append(Action("up", state.swap(i, j, i - 1, j), 1))
                    if i < len(state.board) - 1:
                        actions.append(Action("down", state.swap(i, j, i + 1, j), 1))
                    if j > 0:
                        actions.append(Action("left", state.swap(i, j - 1, i, j), 1))
                    if j < len(state.board) - 1:
                        actions.append(Action("right", state.swap(i, j + 1, i, j), 1))
        return actions

    def is_goal(self, state):
        return state.board == self.final_state.board

    def heuristic(self, state):
        if self.heuristic_type == 'manhattan':
            return self.manhattan_heuristic(state)
        elif self.heuristic_type == 'misplaced':
            return self.misplaced_heuristic(state)

    def manhattan_heuristic(self, state):
        """Manhattan distance heuristic"""
        distance = 0
        for i in range(3):
            for j in range(3):
                value = state.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    distance += abs(i - target_x) + abs(j - target_y)
        return distance

    def misplaced_heuristic(self, state):
        """Misplaced tiles heuristic"""
        misplaced_count = 0
        for i in range(3):
            for j in range(3):
                value = state.board[i][j]
                if value != 0:
                    target_x = (value - 1) // 3
                    target_y = (value - 1) % 3
                    if i != target_x or j != target_y:
                        misplaced_count += 1
        return misplaced_count


def json_to_board(state):
    flat = list(map(lambda n: int(n), state.split(',')))
    return [flat[i:i+3] for i in range(0, len(flat), 3)]


problem = get_problem('sliding_tiles')
pprint.pprint(problem)

initial = SlidingState(json_to_board(problem['initial-state']))
final = SlidingState(json_to_board(problem['final-state']))


print("Breadth-first search result:")
print(breath_first_search(Sliding(initial, final)).to_string(False))

print("\nA* search result with Manhattan heuristic:")
print(A_Star_search(Sliding(initial, final, heuristic_type='manhattan')).to_string(False))

print("\nA* search result wiht Misplaced Tiles heuristic:")
print(A_Star_search(Sliding(initial, final, heuristic_type='misplaced')).to_string(False))


{'explanation': '3x3 grid of tiles, with one missing. Tiles can only move from '
                'an adjacent place to the empty one (0).',
 'final-state': '1,2,3,4,5,6,7,8,0',
 'initial-state': '1,0,2,4,5,3,7,8,6',
 'path-format': '([UDLR](-[UDLR])*)?'}
Breadth-first search result:
Cost: 3
Reached: 18
Branching: 1.8
Time: 0:00:00.000468
Path:
right -> down -> down

A* search result with Manhattan heuristic:
Cost: 3
Reached: 7
Branching: 2.0
Time: 0:00:00.000232
Path:
right -> down -> down

A* search result wiht Misplaced Tiles heuristic:
Cost: 3
Reached: 7
Branching: 2.0
Time: 0:00:00.000192
Path:
right -> down -> down


In [ ]:
#basic labyrinth with heuristic
class LabyrinthState(State):
    def __init__(self, position):
        self.position = position

    def __str__(self):
        return f"Position: {self.position}"

    def __eq__(self, other):
        return self.position == other.position

    def __hash__(self):
        return hash(self.position)

class Labyrinth(Problem):
    def __init__(self, labyrinth_map: str, start: tuple[int, int], end: tuple[int, int]):
        self.labyrinth_map = [list(s) for s in labyrinth_map.split('\n')]
        self.start = start
        self.end = end
        self.map_height = len(self.labyrinth_map)
        self.map_width = len(self.labyrinth_map[0])

    def get_initial_state(self) -> State:
        return LabyrinthState(self.start)

    def actions(self, state) -> list[Action]:
        actions = []
        row, col = state.position

        # Up, Down, Left, Right
        moves = {
            "U": (row - 1, col),
            "D": (row + 1, col),
            "L": (row, col - 1),
            "R": (row, col + 1)
        }

        for move_name, (new_row, new_col) in moves.items():
            if self.is_valid_position(new_row, new_col):
                new_state = LabyrinthState((new_row, new_col))
                actions.append(Action(move_name, new_state, 1))

        return actions

    def is_valid_position(self, row, col) -> bool:
        if 0 <= row < self.map_height and 0 <= col < self.map_width:
            return self.labyrinth_map[row][col] != '#'
        return False

    def heuristic(self, state) -> int:
        row, col = state.position
        goal_row, goal_col = self.end
        return abs(row - goal_row) + abs(col - goal_col)

    def is_goal(self, state) -> bool:
      return state.position == self.end


def find_positions(labyrinth_map) -> tuple[tuple[int, int], tuple[int, int]]:
  """Finding the positions of start 'S' and end 'E' from the map."""
  start_pos = end_pos = None
  for row_index, row in enumerate(labyrinth_map):
      for col_index, char in enumerate(row):
          if char == 'S':
            start_pos = (row_index, col_index)
          elif char == 'E':
            end_pos = (row_index, col_index)
  return start_pos, end_pos


problem = get_problem('labyrinth-big') # here we can change the size of the labyritnh
start_pos, end_pos = find_positions(problem['map'].split('\n'))

print(breath_first_search(Labyrinth(problem['map'], start_pos, end_pos)).to_string(False))
print(A_Star_search(Labyrinth(problem['map'], start_pos, end_pos)).to_string(False))
#pprint.pprint(problem)


Cost: 220
Reached: 4997
Branching: 1.0
Time: 0:00:00.041027
Path:
R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> R -> R -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> L -> L -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> U -> U -> R -> R -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> R -> R -> D -> D -> L -> L -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> R -> R -> R -> R -> U -> U -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> D -> D -> L -> L -> D -> D -> D -> D -> D -> D -> L -> L -> D ->

In [ ]:
#Labyrinth PLUS
class LabyrinthPlusState(State):
    def __init__(self, position, key_found):
        self.position = position
        self.key_found = key_found

    def __str__(self):
        return f"Position: {self.position}, Key found: {self.key_found}"

    def __eq__(self, other):
        return self.position == other.position and self.key_found == other.key_found

    def __hash__(self):
        return hash((self.position, self.key_found))


class LabyrinthPlus(Problem):
    def __init__(self, labyrinth_map: str, start: tuple[int, int], end: tuple[int, int], key_pos: tuple[int, int]):
        self.labyrinth_map = [list(row) for row in labyrinth_map.split('\n')]
        self.start = start
        self.end = end
        self.key_pos = key_pos
        self.map_height = len(self.labyrinth_map[1])
        self.map_width = len(self.labyrinth_map[0])

    def get_initial_state(self) -> State:
        return LabyrinthPlusState(self.start, False)

    def is_goal(self, state) -> bool:
        return state.position == self.end and state.key_found

    def actions(self, state) -> list[Action]:
        actions = []
        row, col = state.position
        key_found = state.key_found

        #moves: Up, Down, Left, Right
        moves = {
            "U": (row - 1, col),
            "D": (row + 1, col),
            "L": (row, col - 1),
            "R": (row, col + 1)
        }

        for move_name, (new_row, new_col) in moves.items():
            if self.is_valid_position(new_row, new_col):
                new_key_found = key_found or (new_row, new_col) == self.key_pos
                new_state = LabyrinthPlusState((new_row, new_col), new_key_found)
                actions.append(Action(move_name, new_state, 1))

        return actions

    def is_valid_position(self, row, col) -> bool:
        """Check if the position is valid (within bounds and not a wall)."""
        if 0 <= row < self.map_height and 0 <= col < self.map_width:
            return self.labyrinth_map[row][col] != '#'
        return False

    def heuristic(self, state) -> int:
      """Only Manhattan distance to the goal is not good enough. So this function is
      first returning distance to the key+goal and then to the goal """
      row, col = state.position
      key_row, key_col = self.key_pos
      goal_row, goal_col = self.end

      if state.key_found:
          return abs(row - goal_row) + abs(col - goal_col)
      else:
          return abs(row - key_row) + abs(col - key_col) + abs(key_row - goal_row) + abs(key_col - goal_col)



def find_positions(labyrinth_map) -> tuple[tuple[int, int], tuple[int, int], tuple[int, int]]:
    """Extract the positions of start 'S', end 'E', and key 'K' from the map."""
    start_pos = end_pos = key_pos = None
    for row_index, row in enumerate(labyrinth_map):
        for col_index, char in enumerate(row):
            if char == 'S':
                start_pos = (row_index, col_index)
            elif char == 'E':
                end_pos = (row_index, col_index)
            elif char == 'K':
                key_pos = (row_index, col_index)
    return start_pos, end_pos, key_pos



labyrinth_map = get_problem('labyrinth-plus-big')

start_pos, end_pos, key_pos = find_positions(labyrinth_map['map'].split('\n'))


print(breath_first_search(LabyrinthPlus(labyrinth_map['map'], start_pos, end_pos, key_pos)).to_string(False))

print(A_Star_search(LabyrinthPlus(labyrinth_map['map'], start_pos, end_pos, key_pos)).to_string(False))
pprint.pprint(labyrinth_map)


Cost: 348
Reached: 9831
Branching: 1.0013240985944185
Time: 0:00:00.147389
Path:
D -> D -> D -> D -> R -> R -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> R -> D -> D -> L -> L -> D -> D -> D -> D -> R -> R -> D -> D -> L -> L -> D -> D -> L -> L -> D -> D -> D -> D -> D -> D -> L -> L -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> D -> D -> R -> R -> D -> D -> D -> D -> D -> D -> D -> D -> L -> L -> L -> L -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> D -> D -> R -> R -> R -> R -> D -> D -> U -> U -> L -> L -> L -> L -> U -> U -> L -> L -> U -> U -> L -> L -> U -> U -> L -> L -> U -> U -> R -> R -> R -> R -> U -> U -> U -> U -> U -> U -> U -> U -> L -> L -> U -> U -> U -> U -> U -> U -> U -> U -> U -> U -> L -> L -> U -> U -> U -> U -> R -> R -> U -> U -> U -> U -> U -> U -> R -> R -> U -> U -> R -> R -> R -> R -> D -> D -> R -> R ->